In [14]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
#import data from google_sheets
mussels= pd.read_csv('https://docs.google.com/spreadsheets/d/10WgzQrIG648U7RA1SBvNdak6cEsb69vH_u_MYcu-I1s/export?format=csv&id')
mussels.shape

In [17]:
#crate new features based on calculated fields
mussels['prizm']= mussels['Length']*mussels['Height']*mussels['Inflation']        
mussels['density']= 1000*mussels['Mass']/(mussels['Length']*mussels['Height']*mussels['Inflation'])                       
mussels['L/H']=mussels.Length/mussels.Height
mussels['L/I']=mussels.Length/mussels.Inflation
mussels['H/I']=mussels.Height/mussels.Inflation
mussels.head()

,Species,Length,Height,Inflation,Mass,prizm,density,L/H,L/I,H/I
0,pistolgrip,127.63,84.4,54.3,NaN,584918.0796,NaN,1.512204,2.350460,1.554328
1,texas pigtoe,88.40,70.8,50.7,NaN,317317.1040,NaN,1.248588,1.743590,1.396450
2,threehorned wartyback,65.90,51.3,42.5,NaN,143678.4750,NaN,1.284600,1.550588,1.207059
3,southern mapleleaf,63.90,56.5,38.0,NaN,137193.3000,NaN,1.130973,1.681579,1.486842
4,southern mapleleaf,63.10,52.6,33.1,NaN,109860.8860,NaN,1.199620,1.906344,1.589124


In [19]:
#drop features with insufficient individuals
mussels_trim=mussels[mussels['Species'].isin(mussels['Species'].value_counts()[mussels['Species'].value_counts()>3].index)]
print (mussels_trim.shape)
mussels_trim.Species.value_counts()

(341, 10)


texas pigtoe             170
southern mapleleaf        84
pistolgrip                27
threehorned wartyback     25
deertoe                    9
threeridge                 7
washboard                  7
western pimpleback         6
louisiana pigtoe           6
Name: Species, dtype: int64

In [34]:
#bokeh Practice
from bokeh.plotting import figure
from bokeh.io import output_file, show
tpt= mussels_trim[mussels_trim.Species=='texas pigtoe']
output_file("line.html")
f=figure()
f.triangle(tpt.Length,tpt.Height,tpt.Inflation)
show(f)

In [35]:
dir(f)

['__cached_all__overridden_defaults__',
 '__cached_all__properties__',
 '__cached_all__properties_with_refs__',
 '__class__',
 '__container_props__',
 '__dataspecs__',
 '__delattr__',
 '__deprecated_attributes__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__overridden_defaults__',
 '__properties__',
 '__properties_with_refs__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__subtype__',
 '__view_model__',
 '__weakref__',
 '_attach_document',
 '_axis',
 '_callbacks',
 '_check_colon_in_category_label',
 '_check_compatible_scale_and_ranges',
 '_check_missing_renderers',
 '_check_no_data_renderers',
 '_check_required_range',
 '_check_required_scale',
 '_check_snapped_toolbar_and_axis',
 '_clone',
 '_detach_document',
 '_document',
 '_event_callbacks',
 '_grid',
 '_id',
 '_ove

In [31]:
X=mussels_trim.drop(['Species','Mass','density'],axis=1)
y=mussels_trim['Species']
print (y.shape, X.shape)

(341,) (341, 7)


In [ ]:
massdf=mussels_trim.dropna(axis=0,how='any')
X_mass=massdf.drop(['Species'],axis=1)
y_mass=massdf['Species']
print (y_mass.shape,X_mass.shape)
massdf.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
y_vect=le.fit_transform(y)
y_mass_vect=le.fit_transform(y_mass)

In [ ]:
from sklearn.decomposition import PCA #Principal component analysis
pca = PCA()
X_PCA = pca.fit(X).transform(X)
X_mass_pca = pca.fit(X).transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_vect, test_size=0.3)
print X_train.shape
print y_train.shape
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_mass, y_mass_vect, test_size=0.3)
print X_train2.shape
print y_train2.shape

KNN classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
model = KNeighborsClassifier(n_neighbors=4)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y_vect)

scores_pca = cross_val_score(model, X_PCA, y_vect)
print(scores)
print(scores_pca)
print("Mean Accuracy:", scores.mean())
print("Mean Accuracy:", scores_pca.mean())

In [ ]:
X_mass = scaler.fit_transform(X_mass)
model = KNeighborsClassifier(n_neighbors=3)
from sklearn.model_selection import cross_val_score
scores_mass = cross_val_score(model, X_mass, y_mass_vect)
print(scores)
print("Mean Accuracy:", scores_mass.mean())
model.fit(X_train,y_train)
pred=model.predict(X_test)

print model.score(X_test,y_test)

In [ ]:
for num_neighbors in range(1, 7):
    # Build the model object and set the number of neighbors required.
    # We will use the default distance metric, Euclidean distance
    model = KNeighborsClassifier(n_neighbors=num_neighbors)

    # Use cross-validation to evaluate the accuracy of the model
    scores = cross_val_score(model, X_mass, y_mass)
    print("Neighbors:", num_neighbors, "Mean Accuracy:", scores.mean())

In [ ]:
scores_mass = cross_val_score(model, X_mass, y_mass_vect)
print(scores)
print("Mean Accuracy:", scores_mass.mean())

Logisitic regression with gridSearch

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
model = GridSearchCV(LogisticRegression(), {'C': [0.001, 0.01, 0.1,0.25,0.3,0.4,0.5,0.6,0.75,
                                                  1.0,1.1,1.2,1.5, 10.0,20],
                                            'penalty': ['l1', 'l2']})
model2 = GridSearchCV(LogisticRegression(), {'C': [0.001, 0.01, 0.1,0.25,0.3,0.4,0.5,0.6,0.75,
                                                  1.0,1.1,1.2,1.5, 10.0,20,50],
                                            'penalty': ['l1', 'l2']})
full=model.fit(X_train, y_train)
mass=model2.fit(X_train2,y_train2)
predy=model.predict(X_test)
print "full:", full.best_params_
print full.best_score_
print "mass: ", mass.best_params_
print mass.best_score_
print model.score(X_test,y_test)

In [ ]:
y_pred = full.predict(X_test)
y_pred2 = mass.predict(X_test2)
#pigtoe = full.predict([54,])
print full.score(X_test,y_pred)
X_test.shape

In [ ]:
cm = confusion_matrix(y_test, y_pred)
idx = y.values
#col = ['Predicted Dead', 'Predicted Survived']
cmdf = pd.DataFrame(cm)#, index=idx)#, columns=col)
cmdf

In [ ]:
cm2 = confusion_matrix(y_test2, y_pred2)
#idx = y.columns
#col = ['Predicted Dead', 'Predicted Survived']
cmdf2 = pd.DataFrame(cm2)#, index=idx, columns=col)
cmdf2

In [ ]:
#silhouette = metrics.silhouette_score(X_test, y_pred, metric='euclidean')

Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth = 7, min_samples_split = 1,criterion='entropy' )
dt.fit(X_train, y_train)
#mass = dt.fit(X_train2, y_train2)
print dt.score(X_train,y_train)

In [ ]:
X=mussels_trim.drop(['Species','Mass','density'],axis=1)
dt.fit(X,y)
print dt.score(X,y)

In [ ]:
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot
from IPython.display import Image
print dt.score(X,y)
dot_data = StringIO()
y_pred = dt.predict(X)
export_graphviz(dt, out_file=dot_data,  
                feature_names=X.columns,  
                class_names=le.classes_,  
                filled=True, rounded=True,
                proportion=False,leaves_parallel=True,
                special_characters=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

In [ ]:
feature_importances = pd.DataFrame(dt.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances.head()

In [ ]:
type(X_mass)

In [ ]:
X_mass=massdf.drop(['Species'],axis=1)
dt.fit(X_train2, y_train2)
print dt.score(X_test2,y_test2)
dot_data = StringIO()
y_pred = dt.predict(X_test2)
export_graphviz(dt, out_file=dot_data,  
                feature_names=X_mass.columns,  
                class_names=le.classes_,  
                filled=True, rounded=True,
                proportion=False,impurity=True,
                special_characters=True,leaves_parallel=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png()) 

In [ ]:
feature_importances = pd.DataFrame(dt.feature_importances_,
                                   index = X_mass.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
#print dt.score(X_test2,y_test2)
#feature_importances.head()

XGBoost 

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
boost = XGBClassifier()
boost.fit(X_train, y_train)
print boost.score(X_test, y_test)

In [ ]:
y_pred_boost = boost.predict(X_test2)
predictions = [round(value) for value in y_pred_boost]
print boost.score(X_test2, y_pred_boost)
cm3 = confusion_matrix(y_test2, y_pred_boost)
#idx = y.columns
#col = ['Predicted Dead', 'Predicted Survived']
cmdf3 = pd.DataFrame(cm3,)#, index=idx, columns=col)
cmdf3

In [ ]:
kfold = StratifiedKFold(n_splits=3, random_state=7)
results = cross_val_score(boost, X_test, y_test, cv=kfold)
print results
print"Accuracy: %.2f%%" % (results.mean()*100)

In [ ]:
print classification_report(y_test2, y_pred)


Random forest and Extra Trees classifiers

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

rf.fit(X_train, y_train)
et.fit(X_train,y_train)

In [ ]:
print rf.score(X_test, y_test)
print et.score(X_test, y_test)

In [ ]:
rf.fit(X_train2, y_train2)
et.fit(X_train2,y_train2)
print rf.score(X_test2, y_test2)
print et.score(X_test2, y_test2)

In [ ]:
import numpy as np
all(rf.feature_importances_ == np.mean([tree.feature_importances_ for tree in rf.estimators_], axis=0))

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

indices = np.argsort(importances)[::-1]
feature_names = X.columns

plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
#plt.show()

In [ ]:
et.fit(X, y)

importances = et.feature_importances_
std = np.std([tree.feature_importances_ for tree in et.estimators_], axis=0)

indices = np.argsort(importances)[::-1]
feature_names = X.columns

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="y", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
#plt.show()

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X, y)

importances = pd.DataFrame(zip(dt.feature_importances_,
                               rf.feature_importances_,
                               et.feature_importances_),
                           index=X.columns,
                           columns=['dt_importance',
                                    'rf_importance',
                                    'et_importance']).sort_values('rf_importance',
                                                                   ascending=False)

print importances.head()
importances.plot(kind='bar')


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
#lda.score(X_r,y)
lda.fit(X_train2,y_train2)
pred=lda.predict(X_test2)
print lda.score(X_test2,y_test2)